# Predicting house prices using k-nearest neighbors regression

In this notebook, you will implement k-nearest neighbors regression. You will:
  * Find the k-nearest neighbors of a given query input
  * Predict the output for the query input using the k-nearest neighbors
  * Choose the best value of k using a validation set

## Imports

In [119]:
# python standard library
import os

# third party
import graphlab
import matplotlib.pyplot as plt
import numpy
from scipy.spatial.distance import euclidean
import seaborn

# this code
import machine_learning
from machine_learning.coursera.regression.common_utilities.numpy_helpers import get_numpy_data
from machine_learning.coursera.regression.common_utilities.numpy_helpers import normalize_features

In [118]:
%matplotlib inline


## Load in house sales data

For this notebook, we use a subset of the King County housing dataset created by randomly selecting 40% of the houses in the full dataset.

In [63]:
data_path = os.path.join(machine_learning.__path__[0],
                         machine_learning.large_data_path,
                         'kc_house_data_small.gl/')
sales = graphlab.SFrame(data_path)

## Import useful functions from previous notebooks

To efficiently compute pairwise distances among data points, we will convert the SFrame into a 2D Numpy array. First import the numpy library and then copy and paste `get_numpy_data()` from the second notebook of Week 2.

Signature: get_numpy_data(data_sframe, features, output)
Docstring:
:param:
 - `data_sframe`: SFrame to convert
 - `features`: list of column names
 - `output`: the target
:return: (matrix of data_sframe columns
          with 'constant' column and 'features' column,
          array from data_sframe using 'output' column)
File:      ~/projects/machine_learning/machine_learning/coursera/regression/common_utilities/numpy_helpers.py
Type:      function

We will also need the `normalize_features()` function from Week 5 that normalizes all feature columns to unit norm. Paste this function below.

Signature: normalize_features(feature_matrix)
Docstring:
:param:
 - `feature_matrix`: numpy array to be normalized (along columns)
:return: (normalized feature_matrix, matrix of norms for feature_matrix)
File:      ~/projects/machine_learning/machine_learning/coursera/regression/common_utilities/numpy_helpers.py
Type:      function

## Split data into training, test, and validation sets

In [64]:
(train_and_validation, test) = sales.random_split(.8, seed=1) # initial train/test split
(train, validation) = train_and_validation.random_split(.8, seed=1) # split training set into training and validation sets

## Extract features and normalize

Using all of the numerical inputs listed in `feature_list`, transform the training, test, and validation SFrames into Numpy arrays:

In [65]:
feature_list = ['bedrooms',  
                'bathrooms',  
                'sqft_living',  
                'sqft_lot',  
                'floors',
                'waterfront',  
                'view',  
                'condition',  
                'grade',  
                'sqft_above',  
                'sqft_basement',
                'yr_built',  
                'yr_renovated',  
                'lat',  
                'long',  
                'sqft_living15',  
                'sqft_lot15']
features_train, output_train = get_numpy_data(train, feature_list, 'price')
features_test, output_test = get_numpy_data(test, feature_list, 'price')
features_valid, output_valid = get_numpy_data(validation, feature_list, 'price')

In [66]:
features_test.shape

(1741, 18)

In computing distances, it is crucial to normalize features. Otherwise, for example, the `sqft_living` feature (typically on the order of thousands) would exert a much larger influence on distance than the `bedrooms` feature (typically on the order of ones). We divide each column of the training feature matrix by its 2-norm, so that the transformed column has unit norm.

IMPORTANT: Make sure to store the norms of the features in the training set. The features in the test and validation sets must be divided by these same norms, so that the training, test, and validation sets are normalized consistently.

In [67]:
features_train, norms = normalize_features(features_train) # normalize training set features (columns)
features_test = features_test / norms # normalize test set by training set norms
features_valid = features_valid / norms # normalize validation set by training set norms

## Compute a single distance

To start, let's just explore computing the "distance" between two given houses.  We will take our **query house** to be the first house of the test set and look at the distance between this house and the 10th house of the training set.

To see the features associated with the query house, print the first row (index 0) of the test feature matrix. You should get an 18-dimensional vector whose components are between 0 and 1.

In [68]:
query_house = features_test[0]

In [69]:
queries = features_test[:3]
queries[0].shape

(18,)

In [70]:
print(query_house)
print(len(features_test[0]))

[ 0.01345102  0.01551285  0.01807473  0.01759212  0.00160518  0.017059    0.
  0.05102365  0.0116321   0.01564352  0.01362084  0.02481682  0.01350306
  0.          0.01345386 -0.01346927  0.01375926  0.0016225 ]
18


Now print the 10th row (index 9) of the training feature matrix. Again, you get an 18-dimensional vector with components between 0 and 1.

In [71]:
print(features_train[9])
print(len(features_train[9]))

[ 0.01345102  0.01163464  0.00602491  0.0083488   0.00050756  0.01279425
  0.          0.          0.01938684  0.01390535  0.0096309   0.
  0.01302544  0.          0.01346821 -0.01346254  0.01195898  0.00156612]
18


***QUIZ QUESTION ***

What is the Euclidean distance between the query house and the 10th house of the training set? 

Note: Do not use the `np.linalg.norm` function; use `np.sqrt`, `np.sum`, and the power operator (`**`) instead. The latter approach is more easily adapted to computing multiple distances at once.

In [72]:
def euclidean_distance(vector_1, vector_2):
    """
    :param:
      - `vector_1': 1-d array
      - `vector_2`: 1-d array
    :return: distance between vector_1 and vector_2
    """
    return numpy.sqrt(numpy.sum((vector_1 - vector_2)**2))

In [73]:
tenth_house = features_train[9]

distance = euclidean_distance(query_house, tenth_house)
print("\nDistance between query house and 10th house: {0:.5f}".format(distance))
assert distance == euclidean(query_house, tenth_house)


Distance between query house and 10th house: 0.05972


## Compute multiple distances

Of course, to do nearest neighbor regression, we need to compute the distance between our query house and *all* houses in the training set.  

To visualize this nearest-neighbor search, let's first compute the distance from our query house (`features_test[0]`) to the first 10 houses of the training set (`features_train[0:10]`) and then search for the nearest neighbor within this small set of houses.  Through restricting ourselves to a small set of houses to begin with, we can visually scan the list of 10 distances to verify that our code for finding the nearest neighbor is working.

Write a loop to compute the Euclidean distance from the query house to each of the first 10 houses in the training set.

*** QUIZ QUESTION ***

Among the first 10 training houses, which house is the closest to the query house?

In [127]:
distance_house = numpy.array([euclidean_distance(query_house, features_train[house]) for house in range(10)])
for house, distance in enumerate(distance_house):
    print("{0}\t{1:.3f}".format(house, distance))
print('closest: {1} ({0:.3f})'.format(distance_house.argmin(), distance_house.min()))
    

0	0.060
1	0.085
2	0.061
3	0.053
4	0.058
5	0.060
6	0.055
7	0.055
8	0.052
9	0.060
closest: 0.052383627841 (8.000)


It is computationally inefficient to loop over computing distances to all houses in our training dataset. Fortunately, many of the Numpy functions can be **vectorized**, applying the same operation over multiple values or vectors.  We now walk through this process.

Consider the following loop that computes the element-wise difference between the features of the query house (`features_test[0]`) and the first 3 training houses (`features_train[0:3]`):

In [75]:
for i in xrange(3):
    print features_train[i]-features_test[0]
    # should print 3 vectors of length 18

[  0.00000000e+00  -3.87821276e-03  -1.20498190e-02  -1.05552733e-02
   2.08673616e-04  -8.52950206e-03   0.00000000e+00  -5.10236549e-02
   0.00000000e+00  -3.47633726e-03  -5.50336860e-03  -2.48168183e-02
  -1.63756198e-04   0.00000000e+00  -1.70072004e-05   1.30577772e-05
  -5.14364795e-03   6.69281453e-04]
[  0.00000000e+00  -3.87821276e-03  -4.51868214e-03  -2.26610387e-03
   7.19763456e-04   0.00000000e+00   0.00000000e+00  -5.10236549e-02
   0.00000000e+00  -3.47633726e-03   1.30705004e-03  -1.45830788e-02
  -1.91048898e-04   6.65082271e-02   4.23240653e-05   6.22415897e-06
  -2.89330197e-03   1.47606982e-03]
[  0.00000000e+00  -7.75642553e-03  -1.20498190e-02  -1.30002801e-02
   1.60518166e-03  -8.52950206e-03   0.00000000e+00  -5.10236549e-02
   0.00000000e+00  -5.21450589e-03  -8.32384500e-03  -2.48168183e-02
  -3.13866046e-04   0.00000000e+00   4.71047219e-05   1.56530415e-05
   3.72914476e-03   1.64764925e-03]


The subtraction operator (`-`) in Numpy is vectorized as follows:

In [76]:
print features_train[0:3] - features_test[0]

[[  0.00000000e+00  -3.87821276e-03  -1.20498190e-02  -1.05552733e-02
    2.08673616e-04  -8.52950206e-03   0.00000000e+00  -5.10236549e-02
    0.00000000e+00  -3.47633726e-03  -5.50336860e-03  -2.48168183e-02
   -1.63756198e-04   0.00000000e+00  -1.70072004e-05   1.30577772e-05
   -5.14364795e-03   6.69281453e-04]
 [  0.00000000e+00  -3.87821276e-03  -4.51868214e-03  -2.26610387e-03
    7.19763456e-04   0.00000000e+00   0.00000000e+00  -5.10236549e-02
    0.00000000e+00  -3.47633726e-03   1.30705004e-03  -1.45830788e-02
   -1.91048898e-04   6.65082271e-02   4.23240653e-05   6.22415897e-06
   -2.89330197e-03   1.47606982e-03]
 [  0.00000000e+00  -7.75642553e-03  -1.20498190e-02  -1.30002801e-02
    1.60518166e-03  -8.52950206e-03   0.00000000e+00  -5.10236549e-02
    0.00000000e+00  -5.21450589e-03  -8.32384500e-03  -2.48168183e-02
   -3.13866046e-04   0.00000000e+00   4.71047219e-05   1.56530415e-05
    3.72914476e-03   1.64764925e-03]]


Note that the output of this vectorized operation is identical to that of the loop above, which can be verified below:

In [77]:
# verify that vectorization works
results = features_train[0:3] - features_test[0]
print results[0] - (features_train[0]-features_test[0])
assert  (results[0] - (features_train[0]-features_test[0])).sum() == 0
# should print all 0's if results[0] == (features_train[0]-features_test[0])
print results[1] - (features_train[1]-features_test[0])
assert (results[1] - (features_train[1]-features_test[0])).sum() == 0
# should print all 0's if results[1] == (features_train[1]-features_test[0])
print results[2] - (features_train[2]-features_test[0])
assert (results[2] - (features_train[2]-features_test[0])).sum() == 0
# should print all 0's if results[2] == (features_train[2]-features_test[0])

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


Aside: it is a good idea to write tests like this cell whenever you are vectorizing a complicated operation.

## Perform 1-nearest neighbor regression

Now that we have the element-wise differences, it is not too hard to compute the Euclidean distances between our query house and all of the training houses. First, write a single-line expression to define a variable `diff` such that `diff[i]` gives the element-wise difference between the features of the query house and the `i`-th training house.

In [78]:
diff = query_house - features_train

To test the code above, run the following cell, which should output a value -0.0934339605842:

In [79]:
last_diff = diff[-1].sum()
tolerance = 1e-14
print(last_diff) # sum of the feature differences between the query and last training house
# should print -0.0934339605842
expected = 0.0934339605842
assert last_diff - 0.0934339605842 < tolerance

0.0934339605842


The next step in computing the Euclidean distances is to take these feature-by-feature differences in `diff`, square each, and take the sum over feature indices.  That is, compute the sum of square feature differences for each training house (row in `diff`).

By default, `np.sum` sums up everything in the matrix and returns a single number. To instead sum only over a row or column, we need to specifiy the `axis` parameter described in the `np.sum` [documentation](http://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.sum.html). In particular, `axis=1` computes the sum across each row.

Below, we compute this sum of square feature differences for all training houses and verify that the output for the 16th house in the training set is equivalent to having examined only the 16th row of `diff` and computing the sum of squares on that row alone.

In [80]:
first = numpy.sum(diff**2, axis=1)[15]
second = numpy.sum(diff[15]**2)
print(first)  # take sum of squares across each row, and print the 16th sum
print(second)  # print the sum of squares for the 16th row -- should be same as above
assert abs(first - second) < tolerance

0.00330705902879
0.00330705902879


With this result in mind, write a single-line expression to compute the Euclidean distances between the query house and all houses in the training set. Assign the result to a variable `distances`.

**Hint**: Do not forget to take the square root of the sum of squares.

In [81]:
distances = numpy.sqrt(numpy.sum((query_house - features_train)**2, axis=1))

To test the code above, run the following cell, which should output a value 0.0237082324496:

In [82]:
actual = distances[100]
print(actual)  # Euclidean distance between the query house and the 101th training house
# should print 0.0237082324496
expected = 0.0237082324496
assert abs(expected - actual) < tolerance


0.0237082324496


Now you are ready to write a function that computes the distances from a query house to all training houses. The function should take two parameters: (i) the matrix of training features and (ii) the single feature vector associated with the query.

In [83]:
def euclidean_two(training, query):
    """
    :param:
     - `training`: matrix of training data
     - `query`: array example to measure distance to
    """
    return numpy.sqrt(numpy.sum((training - query)**2, axis=1))

In [84]:
assert euclidean_two(features_train, query_house)[100] == actual

*** QUIZ QUESTIONS ***

1.  Take the query house to be third house of the test set (`features_test[2]`).  What is the index of the house in the training set that is closest to this query house?
2.  What is the predicted value of the query house based on 1-nearest neighbor regression?

In [85]:
query_three = features_test[2]

In [86]:
distances_three = euclidean_two(features_train, query_three)

In [87]:
closest = distances_three.argmin()
print('Closest House index: {0}, distance: {1:.5f}'.format(closest, distances_three[closest]))

Closest House index: 382, distance: 0.00286


In [88]:
predict_knn_1 = output_train[closest]
print("predicted value of third-house: {0}".format(predict_knn_1))

predicted value of third-house: 249000


## Perform k-nearest neighbor regression

For k-nearest neighbors, we need to find a *set* of k houses in the training set closest to a given query house. We then make predictions based on these k nearest neighbors.

## Fetch k-nearest neighbors

Using the functions above, implement a function that takes in
 * the value of k;
 * the feature matrix for the training houses; and
 * the feature vector of the query house
 
and returns the indices of the k closest training houses. For instance, with 2-nearest neighbor, a return value of [5, 10] would indicate that the 6th and 11th training houses are closest to the query house.

**Hint**: Look at the [documentation for `np.argsort`](http://docs.scipy.org/doc/numpy/reference/generated/numpy.argsort.html).

Signature: numpy.argsort(a, axis=-1, kind='quicksort', order=None)
Docstring:
Returns the indices that would sort an array.

Perform an indirect sort along the given axis using the algorithm specified
by the `kind` keyword. It returns an array of indices of the same shape as
`a` that index data along the given axis in sorted order.

Parameters
----------
a : array_like
    Array to sort.
axis : int or None, optional
    Axis along which to sort.  The default is -1 (the last axis). If None,
    the flattened array is used.
kind : {'quicksort', 'mergesort', 'heapsort'}, optional
    Sorting algorithm.
order : str or list of str, optional
    When `a` is an array with fields defined, this argument specifies
    which fields to compare first, second, etc.  A single field can
    be specified as a string, and not all fields need be specified,
    but unspecified fields will still be used, in the order in which
    they come up in the dtype, to break ties.

Returns
-------
index_array : ndarray, int
    Array of indices that sort `a` along the specified axis.
    In other words, ``a[index_array]`` yields a sorted `a`.


In [89]:
def knn(k, training_data, query_vector):
    """
    :param:
     - `k`: number of neighbors for k-nn
     - `training_data`: matrix with each row representing an instance
     - `query_vector`: array to find nearest neighbors for
    :return: k-tuple of indices of nearest-neighbors
    """
    return euclidean_two(training_data, query_vector).argsort()[:k]

*** QUIZ QUESTION ***

Take the query house to be third house of the test set (`features_test[2]`).  What are the indices of the 4 training houses closest to the query house?

In [90]:
nearest_four = knn(4, features_train, features_test[2])
print("Nearest Four to third house: {0}".format(nearest_four))

Nearest Four to third house: [ 382 1149 4087 3142]


In [91]:
for neighbor in nearest_four:
    print('neighbor: {0}\tdistance{1:.5f}'.format(neighbor, euclidean_distance(features_test[2], features_train[neighbor])))

neighbor: 382	distance0.00286
neighbor: 1149	distance0.00323
neighbor: 4087	distance0.00350
neighbor: 3142	distance0.00359


In [92]:
output_train[nearest_four]

array([249000, 477000, 499950, 430000])

In [93]:
prediction_four = output_train[nearest_four].mean()
print(prediction_four)

413987.5


## Make a single prediction by averaging k nearest neighbor outputs

Now that we know how to find the k-nearest neighbors, write a function that predicts the value of a given query house. **For simplicity, take the average of the prices of the k nearest neighbors in the training set**. The function should have the following parameters:
 * the value of k;
 * the feature matrix for the training houses;
 * the output values (prices) of the training houses; and
 * the feature vector of the query house, whose price we are predicting.
 
The function should return a predicted value of the query house.

**Hint**: You can extract multiple items from a Numpy array using a list of indices. For instance, `output_train[[6, 10]]` returns the prices of the 7th and 11th training houses.

In [94]:
def knn_prediction(k, training, target, query):
    """
    :param:
     - `k`: number of neighbors to poll
     - `training`: feature training matrix of neighbors
     - `target`: array of target output values for each neighbor
     - `query`: array of feature-values to predicto output for
    :return: predicted value for query vector based on average
    """
    neighbors = knn(k, training, query)
    return target[neighbors].mean()

In [95]:
assert prediction_four == knn_prediction(4, features_train, output_train, features_test[2])

*** QUIZ QUESTION ***

Again taking the query house to be third house of the test set (`features_test[2]`), predict the value of the query house using k-nearest neighbors with `k=4` and the simple averaging method described and implemented above.

In [96]:
print("predicted price four third-house based on four neighbors: ${0}".format(prediction_four))

predicted price four third-house based on four neighbors: $413987.5


Compare this predicted value using 4-nearest neighbors to the predicted value using 1-nearest neighbor computed earlier.

In [97]:
print(prediction_four - predict_knn_1)

164987.5


## Make multiple predictions

Write a function to predict the value of *each and every* house in a query set. (The query set can be any subset of the dataset, be it the test set or validation set.) The idea is to have a loop where we take each house in the query set as the query house and make a prediction for that specific house. The new function should take the following parameters:
 * the value of k;
 * the feature matrix for the training houses;
 * the output values (prices) of the training houses; and
 * the feature matrix for the query set.
 
The function should return a set of predicted values, one for each house in the query set.

**Hint**: To get the number of houses in the query set, use the `.shape` field of the query features matrix. See [the documentation](http://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.ndarray.shape.html).

In [100]:
def knn_all(k, training_matrix, target_vector, query_matrix):
    """
    :param:
     - `k`: number of neighbors to compare
     - `training_matrix`: source for the neighbors
     - `target_vector`: vector of target values for prediction
     - `query_matrix`: matrix of vectors to find targets 
    :return: list of predicted values for each query_matrix member
    """
    return [knn_prediction(k, training_matrix, target_vector, query_matrix[query]) for query in range(query_matrix.shape[0])]

*** QUIZ QUESTION ***

Make predictions for the first 10 houses in the test set using k-nearest neighbors with `k=10`. 

1. What is the index of the house in this query set that has the lowest predicted value? 
2. What is the predicted value of this house?

In [106]:
first_ten = features_test[:10]
first_ten_values = knn_all(10, features_train, output_train, first_ten)
lowest = min(first_ten_values)
print(first_ten_values)
print(numpy.where(first_ten_values == lowest))
print(lowest)

[881300.0, 431860.0, 460595.0, 430200.0, 766750.0, 667420.0, 350032.0, 512800.70000000001, 484000.0, 457235.0]
(array([6]),)
350032.0


## Choosing the best value of k using a validation set

There remains a question of choosing the value of k to use in making predictions. Here, we use a validation set to choose this value. Write a loop that does the following:

* For `k` in [1, 2, ..., 15]:
    * Makes predictions for each house in the VALIDATION set using the k-nearest neighbors from the TRAINING set.
    * Computes the RSS for these predictions on the VALIDATION set
    * Stores the RSS computed above in `rss_all`
* Report which `k` produced the lowest RSS on VALIDATION set.

(Depending on your computing environment, this computation may take 10-15 minutes.)

In [107]:
def rss(prediction, actual):
    return ((prediction - actual)**2).sum()

In [109]:
rss_all = [rss(knn_all(k, features_train, output_train, features_valid), output_valid) for k in range(1, 16)]


In [113]:
rss_all = numpy.array(rss_all)
rss_all.shape

(15,)

In [124]:
lowest_k = rss_all.argmin() + 1
lowest_rss = rss_all[lowest_k - 1]
print("Lowest k: {k} (rss {r})".format(k=lowest_k, r=lowest_rss))

Lowest k: 8 (rss 6.73616787355e+13)


To visualize the performance as a function of `k`, plot the RSS on the VALIDATION set for each considered `k` value:

In [123]:
kvals = range(1, 16)
figure, axe = plt.subplots()
lines = axe.plot(kvals, rss_all,'o-')
axe.set_title("RSS vs k")
axe.set_xlabel('k nearest neighbors')
labels = axe.set_ylabel('RSS')


***QUIZ QUESTION ***

What is the RSS on the TEST data using the value of k found above?  To be clear, sum over all houses in the TEST set.

In [125]:
rss(knn_all(lowest_k, features_train, output_train, features_test), output_test)

133118823551516.81